In [1]:
import sys
sys.path.append("/home/ubuntu/MultiModalDeepFake")
import nemo.collections.asr as nemo_asr 
import pandas as pd

/home/ubuntu/.local/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (5.1.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-04-06 22:47:19.466900: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 22:47:19.571936: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-06 22:47:20.104880: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.c

In [2]:
from packages.LJDataLoader import LJDataLoader
from packages.AudioEmbeddingsManager import AudioEmbeddingsManager
from packages.ModelManager import ModelManager
from packages.CadenceModelManager import CadenceModelManager

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



## Generate the Train/Dev/Test

In [22]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
pd.read_csv(file_path)

,Unnamed: 0,id,Main_ID,Secondary_ID,Real,Full_Band_MelGan,HifiGan,MelGan,MelGanLarge,Multi_Band_MelGan,Parallel_WaveGan,Waveglow,ElevenLabs,UberDuck,transcript_1,transcript_2
0,0,LJ001-0001,LJ001,1,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,"Printing, in the only sense with which we are ...","Printing, in the only sense with which we are ..."
1,1,LJ001-0002,LJ001,2,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,in being comparatively modern.,in being comparatively modern.
2,2,LJ001-0003,LJ001,3,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,For although the Chinese took impressions from...,For although the Chinese took impressions from...
3,3,LJ001-0004,LJ001,4,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,"produced the block books, which were the immed...","produced the block books, which were the immed..."
4,4,LJ001-0005,LJ001,5,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,the invention of movable metal letters in the ...,the invention of movable metal letters in the ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13095,13095,LJ050-0274,LJ050,274,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,/home/ubuntu/data/wavefake_data/generated_audi...,made certain recommendations which it believes...,made certain recommendations which it believes...
13096,13096,LJ050-0275,LJ050,275,/home/

In [33]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
loader = LJDataLoader(data_path=file_path)
loader.splitData()

source_architectures = ['Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge', 'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow']
new_col_name = 'RandWaveFake'
loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)

#data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=['RandWaveFake', 'ElevenLabs', 'UberDuck'])
data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=source_architectures+['ElevenLabs', 'UberDuck'])

13055
7833 10444


In [30]:
data_df.head()

,type,id,architecture,path,label,multiclass_label
0,train,LJ003-0180,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
1,train,LJ047-0059,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
2,train,LJ047-0051,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
3,train,LJ003-0244,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
4,train,LJ011-0182,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0


In [34]:
data_df.shape

(130550, 6)

In [32]:
data_df['architecture'].unique()

array(['Real', 'ElevenLabs', 'UberDuck'], dtype=object)

In [6]:
data_df.to_csv('/home/ubuntu/test.csv')

In [7]:
train_count = data_df[data_df['type'] == 'train'].shape[0]
dev_count = data_df[data_df['type'] == 'dev'].shape[0]
test_count = data_df[data_df['type'] == 'test'].shape[0]

print(f'# of Train instances: {train_count}')
print(f'# of Dev instances: {dev_count}')
print(f'# of Test instances: {test_count}')

# of Train instances: 782
# of Dev instances: 262
# of Test instances: 262


## Generate the Features

In [8]:
#speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')
#embedding_manager = AudioEmbeddingsManager(model=speaker_model, data=data_df)
#feature_df, feature_cols = embedding_manager.generateFeatureDf()

In [9]:
cadence_manager = CadenceModelManager(data_df)
feature_df, feature_cols =  cadence_manager.run_cadence_feature_extraction_pipeline()

Normalizing amplitudes
Truncating silences
Truncating audio 100/1306 (8%)
Truncating audio 200/1306 (15%)
Truncating audio 300/1306 (23%)
Truncating audio 400/1306 (31%)
Truncating audio 500/1306 (38%)
Truncating audio 600/1306 (46%)
Truncating audio 700/1306 (54%)
Truncating audio 800/1306 (61%)
Truncating audio 900/1306 (69%)
Truncating audio 1000/1306 (77%)
Truncating audio 1100/1306 (84%)
Truncating audio 1200/1306 (92%)
Truncating audio 1300/1306 (100%)
Extracting pauses
Extracting pause spreads


[NeMo W 2023-04-06 22:48:51 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
      ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
    
[NeMo W 2023-04-06 22:48:51 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
      arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
    
[NeMo W 2023-04-06 22:48:51 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
      ret = ret.dtype.type(ret / rcount)
    
[NeMo W 2023-04-06 22:48:51 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
      return _methods._mean(a, axis=axis, dtype=dtype,
    
[NeMo W 2023-04-06 22:48:51 nemo_logging:349] /home/u

Extracting amplitude features
Creating dataframe
Complete


In [10]:
feature_df.head()

,type,id,architecture,path,label,multiclass_label,pause_ratio,pause_mean,pause_std,n_pauses,amp_deriv,amp_mean
0,train,LJ031-0171,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,0.091974,0.454275,0.727688,0.013605,0.193160,0.512512
1,train,LJ040-0075,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,0.321791,0.038029,0.248401,0.482993,0.013255,0.399403
2,train,LJ034-0068,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,0.069521,0.022478,0.081898,0.197279,0.226004,0.507115
3,train,LJ019-0186,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,0.244059,0.028449,0.145702,0.503401,0.160969,0.349380
4,train,LJ019-0294,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,0.462198,0.087884,0.577696,0.285714,0.046482,0.129883


## Predict and Evaluate

In [11]:
model_manager = ModelManager('svm', feature_df, feature_cols, merge_train_dev=True)

In [12]:
acc, logloss = model_manager.trainPredict(label_col='label')

In [13]:
#model_manager.plotRocCurve()

In [14]:
#model_manager.plotProbaDistribution()

In [15]:
print(acc, logloss)

0.7251908396946565 None
